### Rule-Based Youtube view prediction

### afinn 감성 사전 기반 예측 모델

In [ ]:
import pandas as pd
import numpy as np
from afinn import Afinn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import nltk

In [ ]:
# 1. 데이터 로드
df = pd.read_csv(r"C:\Users\user\Desktop\USvideos.csv")
df = df[['title', 'views']].dropna()
df = df[df['views'] > 0]
df['log_views'] = df['views'].apply(np.log1p)
df = df.head(3000).copy()

# 2. AFINN 감성 분석
afinn = Afinn()
df['afinn_score'] = df['title'].apply(afinn.score)

# 3. 선형 회귀
X = df[['afinn_score']].values
y = df['log_views'].values
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

# 4. 성능 평가
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# 5. 예측값 저장
df['predicted_views'] = np.expm1(y_pred)
df[['title', 'views', 'afinn_score', 'predicted_views']].to_csv("afinn_result.csv", index=False)

### VADER 감성 사전 기반 예측 모델

In [ ]:
# nltk 감성사전 로드
nltk.download('vader_lexicon')

# 1. 데이터 로드
df = pd.read_csv(r"C:\Users\user\Desktop\USvideos.csv")
df = df[['title', 'views']].dropna()
df = df[df['views'] > 0]
df['log_views'] = df['views'].apply(np.log1p)
df = df.head(3000).copy()

# 2. VADER 감성 분석기 생성
vader = SentimentIntensityAnalyzer()

# 3. 제목에 대해 감성 점수 계산 (compound 점수 사용)
df['vader_score'] = df['title'].apply(lambda x: vader.polarity_scores(x)['compound'])

# 4. 선형 회귀 모델 훈련
X = df[['vader_score']].values
y = df['log_views'].values

model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

# 5. 평가 지표 출력
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# 6. 예측값 복원 및 확인
df['predicted_views'] = np.expm1(y_pred)
df[['title', 'views', 'vader_score', 'predicted_views']].to_csv("vader_result.csv", index=False)